# 1 爬虫简介

## 1.1 爬虫是什么

爬虫是什么呢？简单概括：自动从网络上收集信息的一种程序。可能确实比较概括了，说了大家还是不知道这是什么东西。复杂点来说，就是一整套关于数据请求、处理、存储的程序，这之间又涉及到关于网络、数据结构的一些知识。之后的教程也会分别详细介绍关于数据的采集、处理、存储这三方面的知识。

## 1.2 爬虫有什么用
这个问题，直接回答可能不利于大家的想象力。所以让我们来看几个应用场景。
1. 新闻门户网站的新闻共享。比如说今日头条、一点新闻、腾讯新闻等等。他们的新闻来源，一个当然是从自己的记者这里产生，另一部分呢，就是从其他网站上，爬下来他们的记者写的文章。这样才能够做到新闻的门户级别。一个网站报道了一个新闻，其他的网站马上会给你推送相同的新闻了。这中间就是爬虫在起作用，如果是人去找新闻，不仅速度慢，还可能会重复，而且不能24小时工作。这当然就很不方便了。
2. 搜索引擎的一部分。你有没有想过为什么我们能够从百度上搜索到形形色色的网站。为什么百度会存有那些网站的信息，让你一搜就出来了。百度搜索的一部分工作，就是运行它自己的爬虫，从上千万的网站，采集到网页，然后存起来，等待你的搜索。如果没有搜索引擎，你可能会有个小本本，上面记载了你常去的网站的链接，需要去哪个网站了，就拿出小本本翻一下，找一下。
3. 为机器学习，大数据做准备机器学习需要大量的样本，大数据更是需要大量的数据，一般数据直接拿不到，那就只有自己动手从网络上通过爬虫获取了

## 2 HTTP请求

上网的流程基本如下：浏览器输入网址，向本地DNS查询IP，获取IP后发送HTTP请求到指定IP，获取HTTP响应，浏览器解析报文。

爬虫就是代替了浏览器，自动化地进行了查询并解析的功能。所以首先了解一下HTTP协议。

每一个完整的http请求，都由一个request和一个response组成。打开浏览器，按了F12进入开发者模式，打开或者刷新网页，切换到network(网络)就可以看到请求和响应的信息了。

## 2.1 request

也就是我们常说的请求，这是由我们的客户端(浏览器/爬虫)发出的一个查询请求。举个例子：

```
GET /562f25980001b1b106000338.jpg HTTP/1.1
Host    img.mukewang.com
User-Agent    Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36
Accept    image/webp,image/*,*/*;q=0.8
Referer    http://www.imooc.com/
Accept-Encoding    gzip, deflate, sdch
Accept-Language    zh-CN,zh;q=0.8


```

可以见到，request包括的内容有：  
1. 请求行（request line）：用来说明请求类型,要访问的资源以及所使用的HTTP版本.GET说明请求类型为GET,562f25980001b1b106000338.jpg为要访问的资源，该行的最后一部分说明使用的是HTTP1.1版本。
2. 请求头部（header）：紧接着请求行（即第一行）之后的部分，用来说明服务器要使用的附加信息。从第二行起为请求头部，HOST将指出请求的目的地.User-Agent,服务器端和客户端脚本都能访问它,它是浏览器类型检测逻辑的重要基础.该信息由你的浏览器来定义,并且在每个请求中自动发送等等
3. 空行：请求头部后面的空行是必须的
4. 请求数据：通常包含了一些要发送给服务器的数据，这个例子的请求数据为空。

也就是说，其实一般的请求，只需要写前两个内容就可以了。


## 2.2 response

还是以例子来讲解。
```
HTTP/1.1 200 OK
Date: Fri, 22 May 2009 06:07:21 GMT
Content-Type: text/html; charset=UTF-8

<html>
      <head></head>
      <body>
            <!--body goes here-->
      </body>
</html>
```

HTTP响应也由四个部分组成，分别是：状态行、消息报头、空行和响应正文。
1. 状态行，由HTTP协议版本号， 状态码， 状态消息 三部分组成。第一行为状态行，（HTTP/1.1）表明HTTP版本为1.1版本，状态码为200，状态消息为（ok）
2. 消息报头，用来说明客户端要使用的一些附加信息。第二行和第三行为消息报头，Date:生成响应的日期和时间；Content-Type:指定了MIME类型的HTML(text/html),编码类型是UTF-8
3. 空行，消息报头后面的空行是必须的
4. 响应正文，服务器返回给客户端的文本信息。空行后面的html部分为响应正文。

## 2.3 状态码

状态代码有三位数字组成，第一个数字定义了响应的类别，共分五种类别:
* 1xx：指示信息--表示请求已接收，继续处理
* 2xx：成功--表示请求已被成功接收、理解、接受
* 3xx：重定向--要完成请求必须进行更进一步的操作
* 4xx：客户端错误--请求有语法错误或请求无法实现
* 5xx：服务器端错误--服务器未能实现合法的请求

常见状态码：

* 200 OK                        //客户端请求成功
* 400 Bad Request               //客户端请求有语法错误，不能被服务器所理解
* 401 Unauthorized              //请求未经授权，这个状态代码必须和WWW-Authenticate报头域一起使用 
* 403 Forbidden                 //服务器收到请求，但是拒绝提供服务
* 404 Not Found                 //请求资源不存在，eg：输入了错误的URL
* 500 Internal Server Error     //服务器发生不可预期的错误
* 503 Server Unavailable        //服务器当前不能处理客户端的请求，一段时间后可能恢复正常

# 3 网页构成

网页由三部分构成，HTML，CSS和JavaScript，现在这三个内容基本确定，天下网页无出其右。注意JavaScript和Java并没有本质联系，这是两种语言。如果把整个绚丽的网页看成一个人的话，那么html就是这个人的骨肉，css则是好看的外表，JS则能让这个人成长，移动。

教程有限，所以只介绍HTML部分。

## 3.1 HTML简介

HTML全称超文本标记语言，“超文本”就是指页面内可以包含图片、链接，甚至音乐、程序等非文字元素。HTML 不是一种编程语言，而是一种标记语言 (markup language)，使用标记标签来描述网页。

## 3.2 标签

HTML 标记标签通常被称为 HTML 标签 (HTML tag)。

* HTML 标签是由尖括号包围的关键词，比如 \< html\>
* HTML 标签通常是成对出现的，比如 < b> 和 < /b>
* 标签对中的第一个标签是开始标签，第二个标签是结束标签
* 开始和结束标签也被称为开放标签和闭合标签

一个简单的例子就是：
```
<html>
<body>

<h1>我的第一个标题</h1>

<p>我的第一个段落。</p>

</body>
</html>
```
< html> 与 < /html> 之间的文本描述网页  
< body> 与 < /body> 之间的文本是可见的页面内容  
< h1> 与 < /h1> 之间的文本被显示为标题  
< p> 与 < /p> 之间的文本被显示为段落

## 3.3 描述一个标签

标签的名字是在尖括号的，就像每个人的名字一样，而这个人是男是女，长多高，需要用其他的数据去描述。HTML 标签可以拥有属性。属性提供了有关 HTML 元素的更多的信息。并且属性总是以名称/值对的形式出现，比如：name="value"。下面举例子：

HTML 链接由 < a> 标签定义。链接的地址在 href 属性中指定：  
```<a href="http://www.baidu.com">This is a link</a>```  
这里 href=" http://www.baidu.com " 就代表了这个属性。

* ```<h1 align="center">```表示1级标题的文字内容居中
* ```<body bgcolor="yellow">```表示body中的内容的背景颜色是黄色
* ```<a id='25156'>``` 表示a的id编号是25156


# 4 开始着手
## 4.1 获取网页

爬虫的内容就是获取并解析网页，前面铺垫了那么多，其实实现非常简单，因为大部分的工作已经有人帮我们实现了。

首先我们需要用到别人的爬虫库，这里用到了经典的请求库requests。requests.get()返回一个HTTP响应，我们就是针对这个响应进行解析。

In [3]:
import requests
response = requests.get("https://www.baidu.com")
# 状态码
print(response.status_code)
# 消息报头
print(response.headers)
# 响应正文
print(response.content) 

200
{'Cache-Control': 'private, no-cache, no-store, proxy-revalidate, no-transform', 'Connection': 'Keep-Alive', 'Content-Encoding': 'gzip', 'Content-Type': 'text/html', 'Date': 'Fri, 22 Mar 2019 03:11:43 GMT', 'Last-Modified': 'Mon, 23 Jan 2017 13:23:51 GMT', 'Pragma': 'no-cache', 'Server': 'bfe/1.0.8.18', 'Set-Cookie': 'BDORZ=27315; max-age=86400; domain=.baidu.com; path=/', 'Transfer-Encoding': 'chunked'}
b'<!DOCTYPE html>\r\n<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/cache/bdorz/baidu.min.css><title>\xe7\x99\xbe\xe5\xba\xa6\xe4\xb8\x80\xe4\xb8\x8b\xef\xbc\x8c\xe4\xbd\xa0\xe5\xb0\xb1\xe7\x9f\xa5\xe9\x81\x93</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hide

非常简单的几句话，就搞定了。这时候我们得到了网页内容，下面开始解析

## 4.2 解析网页

>Beautiful Soup 是一个可以从HTML或XML文件中提取数据的Python库.它能够通过你喜欢的转换器实现惯用的文档导航,查找,修改文档的方式.Beautiful Soup会帮你节省数小时甚至数天的工作时间.

简单地说，Beautiful Soup就是一个解析器。现在请求和解析都有了，我实在是不知道python的爬虫有什么难的，下面来看看到底有多简单。我们会以 爬取豆瓣电影成都即将上映的影片信息作为案例讲解。


In [6]:
import requests
url = "https://movie.douban.com/cinema/later/chengdu/"
response = requests.get(url)
html=response.content.decode('utf-8')
print(html)

<!DOCTYPE html>
<html lang="zh-cmn-Hans" class="">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="renderer" content="webkit">
    <meta name="referrer" content="always">
    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />
    <title>成都影讯</title>
    
    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />
    <meta http-equiv="Pragma" content="no-cache">
    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">
    
    <meta name="keywords" content="成都,影讯,排片,放映时间,电影票价"/>
    <meta name="description" content="成都影讯,成都电影院排片及放映时间,电影票价查询"/>

    <link rel="apple-touch-icon" href="/pics/movie/apple-touch-icon.png">
    <link href="https://img3.doubanio.com/f/shire/bf61b1fa02f564a4a8f809da7c7179b883a56146/css/douban.css" rel="stylesheet" type="text/css">
    <link href="https://img3.doubanio.com/f/shire/ae3f5a3e3085968370b1fc63afcecb22d3284848/css/separation/_all.css" re

这就是网页的全貌，所有的信息都在里面了，现在只需要用工具把信息提取一下。

这里我们需要告诉解析器，我们要的到底是什么，这一步必须人来做，所以我们需要看懂HTML，注意到这里，所有的即将上映的电影都在这个元素中  
```<div id="showing-soon" class="tab-bd">```  
所以我们先找到这个元素

In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')  # 初始化BeautifulSoup
# print(soup)  # 输出BeautifulSoup转换后的内容
all_movies = soup.find('div', id="showing-soon")  # 先找到最大的div
print(all_movies)  # 输出最大的div的内容

<div class="tab-bd" id="showing-soon">
<div class="item mod ">
<a class="thumb" href="https://movie.douban.com/subject/30164448/">
<img class="" src="https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2551172384.jpg"/>
</a>
<div class="intro">
<h3>
<a class="" href="https://movie.douban.com/subject/30164448/">海市蜃楼</a>
<span class="icon"></span>
</h3>
<ul>
<li class="dt">03月28日</li>
<li class="dt">科幻 / 悬疑 / 惊悚</li>
<li class="dt">西班牙</li>
<li class="dt last"><span class="">26202人想看</span></li>
<a class="trailer_icon" href="https://movie.douban.com/trailer/244371/#content">预告片</a>
</ul>
</div>
</div>
<div class="item mod odd">
<a class="thumb" href="https://movie.douban.com/subject/25924056/">
<img class="" src="https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2549234765.jpg"/>
</a>
<div class="intro">
<h3>
<a class="" href="https://movie.douban.com/subject/25924056/">小飞象</a>
<span class="icon"></span>
</h3>
<ul>
<li class="dt">03月29日</li>
<li class="dt">奇幻 / 冒险</li>


然后我们再来进一步解析，发现电影的完整信息例子如下：
```
<div class="item mod odd">
<a class="thumb" href="https://movie.douban.com/subject/26909787/">
<img class="" src="https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2551252121.jpg"/>
</a>
<div class="intro">
<h3>
<a class="" href="https://movie.douban.com/subject/26909787/">三重威胁之跨国大营救</a>
<span class="icon"></span>
</h3>
<ul>
<li class="dt">03月29日</li>
<li class="dt">动作</li>
<li class="dt">泰国</li>
<li class="dt last"><span class="">2545人想看</span></li>
<a class="trailer_icon" href="https://movie.douban.com/trailer/244575/#content">预告片</a>
</ul>
</div>
</div>
<div class="item mod ">
<a class="thumb" href="https://movie.douban.com/subject/27132728/">
<img class="" src="https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2549893086.jpg"/>
</a>
<div class="intro">
<h3>
<a class="" href="https://movie.douban.com/subject/27132728/">警告</a>
<span class="icon"></span>
</h3>
<ul>
<li class="dt">03月29日</li>
<li class="dt">剧情 / 惊悚</li>
<li class="dt">西班牙</li>
<li class="dt last"><span class="">1409人想看</span></li>
<a class="trailer_icon" href="https://movie.douban.com/trailer/244017/#content">预告片</a>
</ul>
</div>
</div>
```
odd的意思是奇数，这里是说奇数偶数的电影呈现方式会不一样。我们访问页面就可以看到，原来奇数偶数的电影会排在一行。

所以我们知道了所有电影的内容都有个特点，他们的div里面都有item类，也有mod类，奇数的还有odd类。借此可以进行解析了。

In [12]:
movie_list = all_movies.find_all('div', class_="item")  # 从最大的div里面找到影片的div
print(movie_list[0])

<div class="item mod ">
<a class="thumb" href="https://movie.douban.com/subject/30164448/">
<img class="" src="https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2551172384.jpg"/>
</a>
<div class="intro">
<h3>
<a class="" href="https://movie.douban.com/subject/30164448/">海市蜃楼</a>
<span class="icon"></span>
</h3>
<ul>
<li class="dt">03月28日</li>
<li class="dt">科幻 / 悬疑 / 惊悚</li>
<li class="dt">西班牙</li>
<li class="dt last"><span class="">26202人想看</span></li>
<a class="trailer_icon" href="https://movie.douban.com/trailer/244371/#content">预告片</a>
</ul>
</div>
</div>


现在我们得知了每一部电影分开的信息，只需要提取我们想要的就可以了。

这一步还是需要人来做。我们发现：

名字在第 2 个< a>标签里面

链接在第 1 个和第 2 个< a>标签的 href 属性里面

上映日期在第 1 个< li>标签里面

类型在第 2 个< li>标签里面

地区在第 3 个< li>标签里面

关注者数量在第 4 个< li>标签里面


In [14]:
movies_info=[]
for movie in movie_list:
    all_a_tag = movie.find_all('a')  # 找到所有的a标签
    all_li_tag = movie.find_all('li')  # 找到所有的li标签
    movie_name = all_a_tag[1].text  # 从第二个a标签的文字内容提取影片名字
    moive_href = all_a_tag[1]['href']  # 从第二个a标签的文字内容提取影片链接
    movie_date = all_li_tag[0].text  # 从第1个li标签的文字内容提取影片上映时间
    movie_type = all_li_tag[1].text
    movie_area = all_li_tag[2].text
    movie_lovers = all_li_tag[3].text
    movie_info = '名字：{}，链接：{}，日期：{}，类型：{}，地区：{}， 关注者：{}'.format(
        movie_name, moive_href, movie_date, movie_type, movie_area, movie_lovers)
    print(movie_info)
    movies_info.append(movie_info)

名字：海市蜃楼，链接：https://movie.douban.com/subject/30164448/，日期：03月28日，类型：科幻 / 悬疑 / 惊悚，地区：西班牙， 关注者：26202人想看
名字：小飞象，链接：https://movie.douban.com/subject/25924056/，日期：03月29日，类型：奇幻 / 冒险，地区：美国， 关注者：17373人想看
名字：人间·喜剧，链接：https://movie.douban.com/subject/27179414/，日期：03月29日，类型：喜剧，地区：中国大陆， 关注者：13047人想看
名字：三重威胁之跨国大营救，链接：https://movie.douban.com/subject/26909787/，日期：03月29日，类型：动作，地区：泰国， 关注者：2545人想看
名字：警告，链接：https://movie.douban.com/subject/27132728/，日期：03月29日，类型：剧情 / 惊悚，地区：西班牙， 关注者：1409人想看
名字：似水流年，链接：https://movie.douban.com/subject/30384186/，日期：03月29日，类型：剧情，地区：中国大陆， 关注者：671人想看
名字：精灵怪物：疯狂之旅，链接：https://movie.douban.com/subject/27042871/，日期：03月29日，类型：喜剧 / 动画 / 家庭，地区：德国， 关注者：608人想看
名字：毕业旅行之逍遥骑士，链接：https://movie.douban.com/subject/30447810/，日期：04月01日，类型：喜剧，地区：中国大陆， 关注者：159人想看
名字：调音师，链接：https://movie.douban.com/subject/30334073/，日期：04月03日，类型：喜剧 / 犯罪 / 悬疑，地区：印度， 关注者：67862人想看
名字：风中有朵雨做的云，链接：https://movie.douban.com/subject/26728669/，日期：04月04日，类型：剧情 / 犯罪 / 悬疑，地区：中国大陆， 关注者：81627人想看
名字：反贪风暴4，链接：https://movie.douba

现在全部电影的信息都在movies_info里面了。看一下单独电影的表现效果。

In [15]:
movies_info[0]

'名字：海市蜃楼，链接：https://movie.douban.com/subject/30164448/，日期：03月28日，类型：科幻 / 悬疑 / 惊悚，地区：西班牙， 关注者：26202人想看'

如果想要储存，可以用文件操作的教程中的方法。

In [19]:
with open('static/movies.txt','w') as file:
    for movie in movies_info:
        file.write(movie+'\n')

信息已经保存了，现在可以读文件。

In [20]:
with open('static/movies.txt','r') as file:
    print(file.read())

名字：海市蜃楼，链接：https://movie.douban.com/subject/30164448/，日期：03月28日，类型：科幻 / 悬疑 / 惊悚，地区：西班牙， 关注者：26202人想看
名字：小飞象，链接：https://movie.douban.com/subject/25924056/，日期：03月29日，类型：奇幻 / 冒险，地区：美国， 关注者：17373人想看
名字：人间·喜剧，链接：https://movie.douban.com/subject/27179414/，日期：03月29日，类型：喜剧，地区：中国大陆， 关注者：13047人想看
名字：三重威胁之跨国大营救，链接：https://movie.douban.com/subject/26909787/，日期：03月29日，类型：动作，地区：泰国， 关注者：2545人想看
名字：警告，链接：https://movie.douban.com/subject/27132728/，日期：03月29日，类型：剧情 / 惊悚，地区：西班牙， 关注者：1409人想看
名字：似水流年，链接：https://movie.douban.com/subject/30384186/，日期：03月29日，类型：剧情，地区：中国大陆， 关注者：671人想看
名字：精灵怪物：疯狂之旅，链接：https://movie.douban.com/subject/27042871/，日期：03月29日，类型：喜剧 / 动画 / 家庭，地区：德国， 关注者：608人想看
名字：毕业旅行之逍遥骑士，链接：https://movie.douban.com/subject/30447810/，日期：04月01日，类型：喜剧，地区：中国大陆， 关注者：159人想看
名字：调音师，链接：https://movie.douban.com/subject/30334073/，日期：04月03日，类型：喜剧 / 犯罪 / 悬疑，地区：印度， 关注者：67862人想看
名字：风中有朵雨做的云，链接：https://movie.douban.com/subject/26728669/，日期：04月04日，类型：剧情 / 犯罪 / 悬疑，地区：中国大陆， 关注者：81627人想看
名字：反贪风暴4，链接：https://movie.douba

python写爬虫非常简单，教程虽然很长，但是其实真正的时间花在理解背景知识和人工解析HTML上，真正写的代码并不多，总共32行，精简后可能只有20行。

爬虫的强大不仅在于能获取网页的信息，更在于重复利用。这里我们用了```url = "https://movie.douban.com/cinema/later/chengdu/"
```来定位我们想要的信息，但是其实只要网页结构相同，比如北京的即将上映的电影，我们换个url就可以重新运行代码，非常简单，甚至我们可以先获取城市列表，再循环写出每个城市对应的url，再获取全部城市的电影信息。

另外，beautiful soup支持正则表达式，而不仅限于这种指哪儿打哪儿的模式，正则表达式的威力在爬虫中可以得到充分的体现，这里就不展开了。